In [1]:
import pandas as pd
from helpers import get_fibi, get_leumicard, get_isracard, categorize, get_leumi
from datetime import datetime
from ipywidgets import widgets, interact

In [2]:
save_data = pd.read_csv("persistence.csv", index_col=0, parse_dates=True)
dfs = [
    get_fibi("FibiSave201907133433591.xls"),
    get_leumicard("transaction-details_export_1563011498725.xlsx"),
    pd.concat(
        list(
            map(
                get_isracard,
                map(lambda x: "Export_0{}_2019.xls".format(x), range(1, 7)),
            )
        )
    ),
    get_leumi("leumi1.xlsx"),
]

df = pd.concat(dfs, sort=False).reset_index().drop(columns="index")
df = df.sort_values(by="date")
remove = ["הלואה", "הלוואה", "הלוואות", "הלוואות", "תשלום", "וי פיננסי טכנול"]
df = df[~df.name.apply(str).apply(lambda x: any(k in x for k in remove))]
df = df[~df.details.apply(str).apply(lambda x: any(k in x for k in remove))]
# df = df.query("date > @save_data.index.max()")
df["category"] = df.name.apply(categorize)
df = df.drop(columns="details")
pd.DataFrame(df[df.category.isna()].name.unique())

,0
0,העברה מהחשבון
1,2359647 הורא.קבע
2,זיכוי מבל'ל
3,זיכוי מב.המזרחי
4,520669 משיכת שיק
5,520670 משיכת שיק
6,דאר ישראל-שאילתה לטא
7,זיכוי מב.פועלים
8,520671 משיכת שיק
9,ידידי צה'ל בארה


In [3]:
with open("categories.json", encoding="utf-8") as f:
    categories_cache = json.load(f)

In [4]:
def set_category(name, category1, category2, delete=False, all=True, index=None):
    global df
    if delete:
        if all:
            print("deleting all {}".format(name))
            df = df.drop(df.query("name == @name").index)
        else:
            print("deleting row {}: {}".format(index, name))
            df = df.drop(index)
    else:
        categories = categories_cache
        category = category1 or category2
        if category is None:
            raise Exception("None is not legal category")
        if all:
            if categories.get(category) is None:
                categories[category] = []
                print("new category: {}".format(category))
            categories[category].append(name)
            print("{} set to {}".format(category, name))
            df.loc[df.query("name == @name").index, "category"] = category
        else:
            print("{} set to first row: {}".format(category, name))
            df.at[index, "category"] = category

def next_category(name=None, category=None, all=None, index=None):
    if len(df[df.category.isna()]) == 0:
        print("no more categories")
        return
    row = df[df.category.isna()].iloc[0]
    display(
        (df[df[df.category.isna()].iloc[0]["name"] == df["name"]])
        .head()
        .style.hide_index()
    )
    existing_categories = widgets.Dropdown(value=None,
        description="category for", options=sorted(categories_cache.keys())
    )
    new_category = widgets.Text(description="new category", placeholder=row["name"])
    return interact.options(manual=True, manual_name="Set Category")(
        set_category,
        category2=existing_categories,
        category1=new_category,
        name=widgets.fixed(row["name"]),
        all=widgets.Checkbox(value=True, description="All"),
        delete=widgets.Checkbox(value=False, description="Delete"),
        index=widgets.fixed(row.name),
    )

In [5]:
if len(df) == 0:
    raise Exception("No data to add")
latest = df.date.max()
df.to_excel(
    "out/{}.xlsx".format(
        latest.isoformat().split(".")[0].replace("-", "").replace(":", "")
    ),
    index=False,
)
save_data.at[latest, "v"] = True
save_data.to_csv("persistence.csv")

In [7]:
def format_category_json():
    with open("categories.json", mode="r", encoding="utf-8") as f:
        c = json.load(f)
    for k, v in c.items():
        c[k] = sorted(v)
    with open("categories.json", mode="w", encoding="utf-8") as f:
        json.dump(c, f, ensure_ascii=False, sort_keys=True, indent=2)